In [1]:
# Packages

import pandas as pd
import pickle
import glob
import os
import numpy as np
import matplotlib.pyplot as plt
import jax.numpy as jnp
import shutil
import re

In [2]:
k_B = 1.380649e-23 

In [6]:
VEXT_MAX = 10.1
VEXT_MIN = -15
N_BINS   = 20

base_dir = "/Users/danielbock/MASTERTHESIS/MASTA/DataArchiv/Vext_allTEMP"
chem_res = "/Users/danielbock/MASTERTHESIS/MASTA/DataArchiv/bulk_potentials.csv"
pattern  = re.compile(r"Vext_([A-Z]{3})_(\d+)\.pkl")

df = pd.read_csv(chem_res)
pressures = df["pressure_bar"].unique().tolist()

# Falls deine Spalte anders heißt, hier anpassen:
CHEM_COL = "chem_potential_bulk"
if CHEM_COL not in df.columns:
    raise KeyError(f"Spalte '{CHEM_COL}' fehlt in {chem_res}. Vorhanden: {list(df.columns)}")

all_dfs = []
counter = 0

for pressure in pressures:
    for temp_folder in os.listdir(base_dir):
        temp_path = os.path.join(base_dir, temp_folder)
        if not (os.path.isdir(temp_path) and temp_folder.startswith("Vext_")):
            continue
    
        print(f"\nBearbeite Temperatur-Ordner: {temp_folder}")
        rows = []
    
        # alle pickle daten
        for filename in os.listdir(temp_path):
            if not filename.endswith(".pkl"):
                continue
            
            m = pattern.match(filename)
            if not m:
                print(f"Übersprungen (kein gültiger Name): {filename}")
                continue
    
            struct_name, temp = m.groups()
            temp_k = float(temp)  # <-- Temperatur aus dem Namen (NEU explizit int)
            beta = 1/(k_B*temp_k)
            #print(f"Aktuelle temp: {temp_k}")
            file_path = os.path.join(temp_path, filename)
    
            # Entpickeln
            with open(file_path, "rb") as f:
                data = pickle.load(f)
    
            arr = np.asarray(data, dtype=np.float64).squeeze()
            arr = arr[np.isfinite(arr)]  # NaN/Inf entfernen
    
            if arr.size != 64**3:
                counter += 1
    
            # ---------- NEU: chem_res nach Druck & Temperatur filtern und addieren ----------
            sub = df[(df["pressure_bar"] == pressure) & (df["temperature_kelvin"] == temp_k)]
            if not sub.empty:
                chem_val = float(sub[CHEM_COL].mean())   # bei mehreren Treffern: Mittelwert
                arr = arr + chem_val*beta
                print(arr)
            # -------------------------------------------------------------------------------
    
            if arr.size == 0:
                hist = np.zeros(N_BINS, dtype=int)
                edges = np.linspace(0, 1, N_BINS + 1)
            else:
                #arr = np.clip(arr, VEXT_MIN, VEXT_MAX)
                hist, edges = np.histogram(arr, bins=N_BINS)#, range=(VEXT_MIN, VEXT_MAX))
    
            # struktur
            entry = {"structure_name": struct_name, "temperature_kelvin": temp_k, "pressure_bar": pressure}
            entry.update({f"bin_{i}": int(v) for i, v in enumerate(hist)})
            entry["x_max"] = float(edges[-1])
            entry["x_min"] = float(edges[0])
            rows.append(entry)
    
        if rows:
            df_temp = pd.DataFrame(rows)
            all_dfs.append(df_temp)
            print(f"{len(rows)} Strukturen verarbeitet")

# df_all zusammenführen
df_all = pd.concat(all_dfs, ignore_index=True) if all_dfs else pd.DataFrame()
print(f"\n Gesamt-DataFrame mit {len(df_all)} Zeilen erstellt")

# csv
out_name = f"Vext_chem_res_allTEMP_pressure_20b_umrechnen.csv"
out_path = os.path.join(base_dir, out_name)
df_all.to_csv(out_path, index=False)

print(f"Datei gespeichert unter:\n{out_path}")



Bearbeite Temperatur-Ordner: Vext_400
[nan nan nan ... nan nan nan]


/var/folders/p_/gl5tyfs92yg7lps1mzt8whdc0000gn/T/ipykernel_20567/1354086475.py:59: RuntimeWarning: invalid value encountered in sqrt
  arr = arr + np.sqrt(chem_val*beta)


ValueError: autodetected range of [nan, nan] is not finite

Vext HIST: Daten Enpickeln und Histogrammieren

In [3]:
VEXT_MAX = 10.1         # 10.1
VEXT_MIN = -15  # -15 unten global "-14.744"
N_BINS   = 100

base_dir = "/Users/danielbock/MASTERTHESIS/MASTA/DataArchiv/Vext_allTEMP"
chem_res = "/Users/danielbock/MASTERTHESIS/MASTA/DataArchiv/bulk_potentials.csv"
pattern  = re.compile(r"Vext_([A-Z]{3})_(\d+)\.pkl")

df = pd.read_csv(chem_res)
pressures = df["pressure_bar"].unique().tolist()
CHEM_COL = "chem_potential_bulk" 

all_dfs = []
counter = 0

for pressure in pressures:
    for temp_folder in os.listdir(base_dir):
        temp_path = os.path.join(base_dir, temp_folder)
        if not (os.path.isdir(temp_path) and temp_folder.startswith("Vext_")):
            continue
    
        print(f"\nBearbeite Temperatur-Ordner: {temp_folder}")
        rows = []
    
        # alle pickle daten
        for filename in os.listdir(temp_path):
            if not filename.endswith(".pkl"):
                continue
            
            m = pattern.match(filename)
            if not m:
                print(f"Übersprungen (kein gültiger Name): {filename}")
                continue
    
            struct_name, temp = m.groups()
            file_path = os.path.join(temp_path, filename)
    
            # Entpickeln
            with open(file_path, "rb") as f:
                data = pickle.load(f)
    
            arr = np.asarray(data, dtype=np.float64).squeeze()
            arr = arr[np.isfinite(arr)]  # NaN/Inf entfernen
    
            if arr.size != 64**3:
                counter += 1
            #print(counter)
    
            # Mask und exp(-)
            if arr.size == 0:
                hist = np.zeros(N_BINS, dtype=int)
                edges = np.linspace(0, 1, N_BINS + 1)
            else:
                arr = np.clip(arr, VEXT_MIN, VEXT_MAX)
                # variante noexp / linBins
                #hist, edges = np.histogram(arr, bins=N_BINS, range=(VEXT_MIN, VEXT_MAX))
                # variante noexp / logBins
                #edges = np.logspace(VEXT_MIN, VEXT_MAX, N_BINS + 1)
                #hist, edges = np.histogram(arr, bins=edges) # automatisch äquistante bins
                # variante exp / logBins
                #edges = np.logspace(np.log(np.exp(-VEXT_MAX)), np.log(np.exp(-VEXT_MIN)), N_BINS + 1)
                hist, edges = np.histogram(np.exp(-arr), bins=N_BINS, range=(0, np.exp(10))) # automatisch äquistante bins
                #clipped
                #
                # variante 
    
    
            # struktur
            entry = {"structure_name": struct_name, "temperature_kelvin": int(temp)}
            entry.update({f"bin_{i}": int(v) for i, v in enumerate(hist)})
            entry["x_max"] = float(edges[-1])
            entry["x_min"] = float(edges[0])
            rows.append(entry)
    
        # -
        if rows:
            df_temp = pd.DataFrame(rows)
            all_dfs.append(df_temp)
            print(f"{len(rows)} Strukturen verarbeitet")

# df_all zusammenführen
df_all = pd.concat(all_dfs, ignore_index=True) if all_dfs else pd.DataFrame()
print(f"\n Gesamt-DataFrame mit {len(df_all)} Zeilen erstellt")

# csv
#out_name = f"Vext_allTEMP_exp_rangelogBin_{VEXT_MAX}_{VEXT_MIN}_{N_BINS}Bins.csv"
out_name = f"Vext_allTEMP_64grid_100b_exp.csv"
out_path = os.path.join(base_dir, out_name)
#df_all.to_csv(out_path, index=False)

print(f"Datei gespeichert unter:\n{out_path}")


Bearbeite Temperatur-Ordner: Vext_400
Vext_400
/Users/danielbock/MASTERTHESIS/MASTA/DataArchiv/Vext_allTEMP/Vext_400
Vext_JSW_400.pkl
Vext_400
/Users/danielbock/MASTERTHESIS/MASTA/DataArchiv/Vext_allTEMP/Vext_400
Vext_SZR_400.pkl
Vext_400
/Users/danielbock/MASTERTHESIS/MASTA/DataArchiv/Vext_allTEMP/Vext_400
Vext_BPH_400.pkl
Vext_400
/Users/danielbock/MASTERTHESIS/MASTA/DataArchiv/Vext_allTEMP/Vext_400
Vext_USI_400.pkl
Vext_400
/Users/danielbock/MASTERTHESIS/MASTA/DataArchiv/Vext_allTEMP/Vext_400
Vext_THO_400.pkl
Vext_400
/Users/danielbock/MASTERTHESIS/MASTA/DataArchiv/Vext_allTEMP/Vext_400
Vext_LAU_400.pkl
Vext_400
/Users/danielbock/MASTERTHESIS/MASTA/DataArchiv/Vext_allTEMP/Vext_400
Vext_ETR_400.pkl
Vext_400
/Users/danielbock/MASTERTHESIS/MASTA/DataArchiv/Vext_allTEMP/Vext_400
Vext_TSC_400.pkl
Vext_400
/Users/danielbock/MASTERTHESIS/MASTA/DataArchiv/Vext_allTEMP/Vext_400
Vext_OFF_400.pkl
Vext_400
/Users/danielbock/MASTERTHESIS/MASTA/DataArchiv/Vext_allTEMP/Vext_400
Vext_FRA_400.pkl
V

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# --- Einstellungen ---
csv_files = [
    "/Users/danielbock/MASTERTHESIS/MASTA/DataArchiv/Vext_allTEMP/Vext_allTEMP_exp_rangelogBin_10.1_-15_100Bins.csv",
    "/Users/danielbock/MASTERTHESIS/MASTA/DataArchiv/Vext_allTEMP/Vext_allTEMP_noexp_rangelinBin_10.1_-15_100Bins.csv",
    "/Users/danielbock/MASTERTHESIS/MASTA/DataArchiv/Vext_allTEMP/Vext_allTEMP_noexp_rangelogBin_10.1_-15_100Bins.csv",
    "/Users/danielbock/MASTERTHESIS/MASTA/DataArchiv/Vext_allTEMP/Vext_allTEMP_exp_rangelogBin_10.1_-15_100Bins_test.csv",
]

structure = "MWW"
temperature = 500   # gewünschte Temperatur in K
pressure = None     # z. B. 1.0, falls vorhanden

# --- Schleife über alle CSVs ---
for file in csv_files:
    if not os.path.exists(file):
        print(f"⚠️ Datei nicht gefunden: {file}")
        continue

    df = pd.read_csv(file)

    # Spaltennamen automatisch erkennen
    structure_col = "structure_name" if "structure_name" in df.columns else "structure"
    temp_col = "temperature_kelvin" if "temperature_kelvin" in df.columns else "temperature"

    # Filter nach Struktur, Temperatur und ggf. Druck
    if pressure is not None and "Pressure_bar" in df.columns:
        df_sel = df[
            (df[structure_col] == structure) &
            (df[temp_col] == temperature) &
            (np.isclose(df["Pressure_bar"], pressure, atol=1e-3))
        ]
    else:
        df_sel = df[
            (df[structure_col] == structure) &
            (df[temp_col] == temperature)
        ]

    if df_sel.empty:
        print(f"⚠️ Keine Daten für Struktur {structure} in {os.path.basename(file)} gefunden.")
        continue

    # Falls mehrere Zeilen vorhanden sind, nimm die erste
    row = df_sel.iloc[0]

    # Histogrammdaten extrahieren
    bin_cols = [c for c in df.columns if c.startswith("bin_")]
    y = row[bin_cols].values
    x_min, x_max = row["x_min"], row["x_max"]

    # X-Achse rekonstruieren
    x_edges = np.linspace(x_min, x_max, len(bin_cols) + 1)
    x_centers = 0.5 * (x_edges[:-1] + x_edges[1:])

    # --- Plot ---
    plt.figure(figsize=(8, 5))
    plt.plot(x_centers, y, marker="o", lw=1.5)
    plt.grid(True, alpha=0.4)

    plt.title(f"{structure} – {int(temperature)} K\n{os.path.basename(file)}")
    plt.yscale('log')
    plt.xlabel("Vext")
    plt.ylabel("Häufigkeit pro Bin")
    plt.tight_layout()
    plt.show()



In [ ]:
VEXT_MAX = 10.0
VEXT_MIN = -15.0
N_BINS   = 500

base_dir = "/Users/danielbock/MASTERTHESIS/MASTA/DataArchiv/Vext_allTEMP"
pattern  = re.compile(r"Vext_([A-Z]{3})_(\d+)\.pkl")

# --- Alle Vext-Werte sammeln ---
all_values = []
counter = 0

for temp_folder in os.listdir(base_dir):
    temp_path = os.path.join(base_dir, temp_folder)
    if not (os.path.isdir(temp_path) and temp_folder.startswith("Vext_")):
        continue

    print(f"Bearbeite Temperatur-Ordner: {temp_folder}")

    for filename in os.listdir(temp_path):
        if not filename.endswith(".pkl"):
            continue

        m = pattern.match(filename)
        if not m:
            print(f"Übersprungen (ungültiger Name): {filename}")
            continue

        file_path = os.path.join(temp_path, filename)

        with open(file_path, "rb") as f:
            data = pickle.load(f)

        arr = np.asarray(data, dtype=np.float64).squeeze()
        arr = arr[np.isfinite(arr)]

        if arr.size != 64**3:
            counter += 1
            print(f"{filename} hat unerwartete Größe {arr.size}")
            continue

        # Werte clippen (damit Extremwerte das Histogramm nicht verzerren)
        arr = np.clip(arr, VEXT_MIN, VEXT_MAX)
        all_values.append(arr)

# --- Alle Werte zu einem Array verbinden ---
if len(all_values) == 0:
    raise ValueError("Keine Daten gefunden!")

all_values = np.concatenate(all_values)
print(f" Gesamtzahl Werte: {len(all_values):,}")

# --- Histogramm ---
hist, edges = np.histogram(all_values, bins=N_BINS, range=(VEXT_MIN, VEXT_MAX))

#print("Fertig! Histogramm über alle Strukturen geplottet.")

In [ ]:
# --- Plot ---
plt.figure(figsize=(8, 5))
plt.plot(0.5*(edges[1:]+edges[:-1]), hist, lw=1.5)
plt.title(f"Histogramm aller Strukturen ({len(all_values):,} Werte)")
plt.xlabel("Vext")
plt.ylabel("Anzahl")
plt.ylim(0, 1500000)
plt.grid(True, alpha=0.5)
plt.tight_layout()
plt.show()

HIST TEST - NO CSV

In [ ]:
# Packages

import pandas as pd
import pickle
import glob
import os
import numpy as np
import matplotlib.pyplot as plt
import jax.numpy as jnp
import shutil
import re
from scipy.spatial import distance

N_BINS=200
VEXT_MIN=-10
VEXT_MAX=5

file_path1 = "/Users/danielbock/MASTERTHESIS/MASTA/DataArchiv/Vext_allTEMP/Vext_500/Vext_EON_500.pkl"
file_path2 = "/Users/danielbock/MASTERTHESIS/MASTA/DataArchiv/Vext_allTEMP/Vext_500/Vext_MWW_500.pkl"

with open(file_path1, "rb") as f:
    data = pickle.load(f)

with open(file_path2, "rb") as f:
    doto = pickle.load(f)


In [ ]:
"""
arr1 = np.asarray(np.exp(-data), dtype=np.float64).squeeze()
arr1 = arr1[np.isfinite(arr1)]
arr2 = np.asarray(np.exp(-doto), dtype=np.float64).squeeze()
arr2 = arr2[np.isfinite(arr2)]  # NaN/Inf entfernen

""" #betaVext und -betaVext
arr1 = np.asarray(data, dtype=np.float64).squeeze()
arr1 = arr1[np.isfinite(arr1)]
arr2 = np.asarray(doto, dtype=np.float64).squeeze()
arr2 = arr2[np.isfinite(arr2)]  # NaN/Inf entfernen

arr1 = np.clip(arr1, VEXT_MIN, VEXT_MAX)
arr2 = np.clip(arr2, VEXT_MIN, VEXT_MAX)


In [ ]:
""" 
hist1, edges1 = np.histogram(np.exp(arr1), bins=np.logspace(0, (10), N_BINS))#, range=(VEXT_MIN, VEXT_MAX))
hist2, edges1 = np.histogram(np.exp(arr2), bins=np.logspace(0, (10), N_BINS))#, range=(VEXT_MIN, VEXT_MAX))

hist11, edges11 = np.histogram(np.exp(-arr1), bins=np.logspace(0, (10), N_BINS))#, range=(VEXT_MIN, VEXT_MAX))
hist22, edges22 = np.histogram(np.exp(-arr2), bins=np.logspace(0, (10), N_BINS))#, range=(VEXT_MIN, VEXT_MAX))


hist1, edges1 = np.histogram(np.exp(arr1), bins=N_BINS, range=(VEXT_MIN, VEXT_MAX))
hist2, edges1 = np.histogram(np.exp(arr2), bins=N_BINS, range=(VEXT_MIN, VEXT_MAX))
hist11, edges11 = np.histogram(np.exp(-arr1), bins=N_BINS, range=(VEXT_MIN, VEXT_MAX))
hist22, edges22 = np.histogram(np.exp(-arr2), bins=N_BINS, range=(VEXT_MIN, VEXT_MAX))
"""
hist1, edges1 = np.histogram(arr1, bins=N_BINS, range=(VEXT_MIN, VEXT_MAX))
hist2, edges2 = np.histogram(arr2, bins=N_BINS, range=(VEXT_MIN, VEXT_MAX))
hist11, edges11 = np.histogram((arr1), bins=N_BINS, range=(VEXT_MIN, VEXT_MAX))
hist22, edges22 = np.histogram((arr2), bins=N_BINS, range=(VEXT_MIN, VEXT_MAX))

x_edges1 = 0.5 * (edges1[:-1] + edges1[1:])
x_edges2 = 0.5 * (edges11[:-1] + edges11[1:])  # für die negierten

# --- Plots ---
plt.figure(figsize=(10, 6))

# Original-Histogramme
plt.plot(x_edges1, hist1, label="(arr1)", lw=1.5)
plt.plot(x_edges1, hist2, label="(arr2)", lw=1.5, linestyle="--")

# Negierte Histogramme
#plt.plot(x_edges2, hist11, label="(-arr1)", lw=1.5)
#plt.plot(x_edges2, hist22, label="(-arr2)", lw=1.5, linestyle="--")

# --- Darstellung ---
plt.title("Vergleich der Histogramme (arr1, arr2 und negierte Varianten)")
plt.xlabel("Vext")
plt.ylabel("Häufigkeit pro Bin")
#plt.xlim(, 25000)
plt.grid(True, alpha=0.4)
plt.yscale('log')
plt.xscale('linear')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
print(np.sum(hist1))
print(np.sum(hist2))

